# Build Validation Table for NPMRDS
For each of the 9 SACSIM time periods, provide:
* Average speed
* Number of epochs
* Standard error of the speed (standard deviation / mean)

In [12]:
import os
from time import perf_counter as perf

import pandas as pd
import urllib
import sqlalchemy as sqla # needed to run pandas df.to_sql() function
    
# extract SQL Server query results into a pandas dataframe   
def sqlqry_to_df(query_str, dbname, servername='SQL-SVR', trustedconn='yes'):     

    conn_str = "DRIVER={ODBC Driver 17 for SQL Server};" \
        f"SERVER={servername};" \
        f"DATABASE={dbname};" \
        f"Trusted_Connection={trustedconn}"
        
    conn_str = urllib.parse.quote_plus(conn_str)
    engine = sqla.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")
       
    start_time = perf()

    # create SQL table from the dataframe
    print("Executing query. Results loading into dataframe...")
    df = pd.read_sql_query(sql=query_str, con=engine)
    rowcnt = df.shape[0]
    
    et_mins = round((perf() - start_time) / 60, 2)
    print(f"Successfully executed query in {et_mins} minutes. {rowcnt} rows loaded into dataframe.")
    
    return df

def get_prd_df(qry_file, db_name, speed_tbl, tmc_tbl, prd_tag, prd_hrs):
    
    with open(qry_file, 'r') as f:
        q_str = f.read()
        
    q_str_formatted = q_str.format(speed_tbl, prd_hrs, prd_tag, tmc_tbl)
    
    prd_df = sqlqry_to_df(q_str_formatted, db_name)
    
    return prd_df
    

In [13]:
query_file = 'cong_speed_model_prds.sql'

db = 'NPMRDS'
tbl_speeds = 'npmrds_2016_alltmc_paxtruck_comb'
tbl_tmcs = 'npmrds_2020_alltmc_txt'
join_key = 'tmc'

prds = {'h07':[7], 'h08':[8], 'h09':[9], 'md5':[10, 11, 12, 13, 14], 
        'h15':[15], 'h16':[16], 'h17':[17], 'ev2':[18, 19], 
        'n11': [20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6]}

df_master = pd.DataFrame()

for i, prdtag in enumerate(prds.keys()):
    print(f"loading in data for {prdtag}...")
    prd_hours = ','.join(str(i) for i in prds[prdtag])
    if i == 0:
        df_master = get_prd_df(qry_file=query_file, db_name=db,
                               speed_tbl=tbl_speeds, tmc_tbl=tbl_tmcs, 
                               prd_tag=prdtag, prd_hrs=prd_hours)
    else:
        df_p = get_prd_df(qry_file=query_file, db_name=db,
                               speed_tbl=tbl_speeds, tmc_tbl=tbl_tmcs, 
                               prd_tag=prdtag, prd_hrs=prd_hours)
        
        df_master = df_master.merge(df_p, on=join_key)
        
        
print("successfully created dataframe for speeds in all sacsim time periods.")
        

    


loading in data for h07...
Executing query. Results loading into dataframe...
Successfully executed query in 0.21 minutes. 7495 rows loaded into dataframe.
loading in data for h08...
Executing query. Results loading into dataframe...
Successfully executed query in 0.21 minutes. 7495 rows loaded into dataframe.
loading in data for h09...
Executing query. Results loading into dataframe...
Successfully executed query in 0.22 minutes. 7495 rows loaded into dataframe.
loading in data for md5...
Executing query. Results loading into dataframe...
Successfully executed query in 0.36 minutes. 7495 rows loaded into dataframe.
loading in data for h15...
Executing query. Results loading into dataframe...
Successfully executed query in 0.21 minutes. 7495 rows loaded into dataframe.
loading in data for h16...
Executing query. Results loading into dataframe...
Successfully executed query in 0.21 minutes. 7495 rows loaded into dataframe.
loading in data for h17...
Executing query. Results loading into

In [14]:
df_master.iloc[0]

tmc               105-16985
avspd_h07         22.278737
epcnt_h07                17
spd_stderr_h07     0.869202
avspd_h08         17.379685
epcnt_h08                36
spd_stderr_h08     0.439095
avspd_h09         23.116691
epcnt_h09                 5
spd_stderr_h09     0.253771
avspd_md5         18.061921
epcnt_md5                60
spd_stderr_md5      0.37128
avspd_h15         14.894814
epcnt_h15                26
spd_stderr_h15     0.608892
avspd_h16         20.805381
epcnt_h16                23
spd_stderr_h16     0.351556
avspd_h17         16.132635
epcnt_h17                14
spd_stderr_h17     1.342916
avspd_ev2         25.678396
epcnt_ev2                20
spd_stderr_ev2     0.326428
avspd_n11         21.661711
epcnt_n11                13
spd_stderr_n11     0.376075
Name: 0, dtype: object